In [2]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context("talk")
sns.set_style("white")

In [18]:
marita_df = pd.read_csv("marita_results_2021-04-16.csv")
marita_condensed = marita_df[['scenario_number', 'archetype', 'drug_type', 'Scenario',
       'cost.CHW', 'cost.RDT', 'cost.drug', 'cost.ACT', 'cost.smc',
       'cost.facility', 'cost.severe', 'cost.bednet','cost.smc.ref', 'cost.smc.low', 'cost.low',
       'cost', 'cases']]
marita_condensed = marita_condensed[marita_condensed["archetype"]=="Sahel"].sort_values(by="Scenario").reset_index(drop=True)

In [4]:
def get_agg_df_w_baseline(archetype):
    def _add_drug_length(df):
        df["drug_length"] = -1
        df.loc[df["drug_type"]=="DP", "drug_length"] = 28
        df.loc[np.logical_or(df["drug_type"]=="ASAQ", df["drug_type"]=="SPAQ"), "drug_length"] = 14
        
    scenario_df = pd.read_csv("../scenario_master_list.csv")
    scenario_df = scenario_df[scenario_df["archetype"]==archetype].reset_index(drop=True)

    if archetype == "Sahel":
#         df = pd.read_csv("endpoints_bdc41c37-b09c-eb11-a2ce-c4346bcb1550.csv")
#         df.drop(df[np.in1d(df['scenario_number'], [35, 36])].index, inplace = True)
#         df2 = pd.read_csv("endpoints_c1c725ad-e49c-eb11-a2ce-c4346bcb1550.csv") # has corrected scenarios 35 and 36
#         df = pd.concat([df,df2])
#         df = pd.read_csv("endpoints_1e270e7a-559d-eb11-a2ce-c4346bcb1550.csv")
        df = pd.read_csv("endpoints_312b5201-609d-eb11-a2ce-c4346bcb1550.csv")
        df2 = pd.read_csv("endpoints_44bed067-43a2-eb11-a2ce-c4346bcb1550.csv") # SMC with DP
        df = pd.concat([df,df2])
    elif archetype == "Southern":
#         df = pd.read_csv("endpoints_f70751f3-2d98-eb11-a2ce-c4346bcb1550.csv") 
#         df2 = pd.read_csv("endpoints_b925ffe3-af9c-eb11-a2ce-c4346bcb1550.csv")
#         df = pd.concat([df, df2])
        df = pd.read_csv("endpoints_a71e7b31-559d-eb11-a2ce-c4346bcb1550.csv")
        df2 = pd.read_csv("endpoints_b7cd3df7-24a2-eb11-a2ce-c4346bcb1550.csv") #extra ITN scenario
        df = pd.concat([df,df2])
    

    # df cleaning
    df["out_of_school_rate"] = np.round(df["out_of_school_rate"],2)
    _add_drug_length(df)
    df.fillna({"smc_on": False,
                "smc_age_range": "default",
                "ivermectin": False,
                "primaquine": False}, inplace=True)
    
    df_baseline = df[df["scenario_number"]==30].groupby("burnin_approx_pfpr2_10").agg("mean").reset_index()
#     df_allout = df[df["scenario_number"]==27].groupby("burnin_approx_pfpr2_10").agg("mean").reset_index()

    df_agg = df.groupby(["burnin_approx_pfpr2_10", "scenario_number"]).agg("mean").reset_index().merge(scenario_df, on="scenario_number", suffixes=("","_DUPLICATED"))
    df_agg["out_of_school_rate"] = np.round(df_agg["out_of_school_rate"],2)

    df_agg_w_baseline = df_agg.merge(df_baseline, on="burnin_approx_pfpr2_10", suffixes=["","_BASELINE"], how="left")
    
    return df_agg_w_baseline

In [5]:
foo = get_agg_df_w_baseline("Sahel")

# u10 SMC

In [54]:
df_u5 = foo[foo["scenario_number"]==30].reset_index(drop=True)
df_u10 = foo[foo["scenario_number"]==35].reset_index(drop=True)
df_u15 = foo[foo["scenario_number"]==36].reset_index(drop=True)
df_u10_dp = foo[foo["scenario_number"]==46].reset_index(drop=True)
df_u15_dp = foo[foo["scenario_number"]==47].reset_index(drop=True)
print(np.mean(df_u5["Received_SMC"]))
print(np.mean(df_u10["Received_SMC"]))
print(np.mean(df_u15["Received_SMC"]))

df_u10_dp["case_ratio_to_old"] = df_u10_dp["clinical_incidence_all"]/df_u10["clinical_incidence_all"]
df_u10_dp["severe_case_ratio_to_old"] = df_u10_dp["severe_incidence_all"]/df_u10["severe_incidence_all"]

6590.175
10378.949999999999
14450.675


In [74]:
(df_u5["clinical_incidence_all"]-df_u10_dp["clinical_incidence_all"])/df_u5["clinical_incidence_all"]

0    0.223903
1    0.228085
2    0.209728
3    0.187793
Name: clinical_incidence_all, dtype: float64

In [62]:
df_u10_dp["clinical_incidence_all"]

0    0.740739
1    0.902358
2    0.995806
3    1.070668
Name: clinical_incidence_all, dtype: float64

In [63]:
df_u10["clinical_incidence_all"]

0    0.822951
1    1.012718
2    1.088046
3    1.141978
Name: clinical_incidence_all, dtype: float64

In [42]:
ipt_cost = marita_condensed[marita_condensed["scenario_number"]==13].reset_index(drop=True)
spaq_smc_cost = marita_condensed[marita_condensed["scenario_number"]==35].reset_index(drop=True)
dp_smc_cost = spaq_smc_cost.copy(deep=True)

In [43]:
dp_smc_cost = dp_smc_cost.merge(spaq_smc_cost, on="Scenario", suffixes=("_DP", "_SPAQ"))

> I assumed the SMC cost for the u10 and u15 scenarios had SMC cost calculated as: smc cost from scenario 30 (SMC alone) + (number received drugs in SMC - number received drugs in SMC in scenario 30) * drug cost --Marita

In [70]:
dp_smc_cost["scenario_number_DP"] = 46
dp_smc_cost["cost.RDT_DP"] = dp_smc_cost["cost.RDT_SPAQ"]*df_u10_dp["case_ratio_to_old"]
dp_smc_cost["cost.ACT_DP"] = dp_smc_cost["cost.ACT_SPAQ"]*df_u10_dp["case_ratio_to_old"]
dp_smc_cost["cost.facility_DP"] = dp_smc_cost["cost.facility_SPAQ"]*df_u10_dp["case_ratio_to_old"]
dp_smc_cost["cost.severe_DP"] = dp_smc_cost["cost.severe_SPAQ"]*df_u10_dp["severe_case_ratio_to_old"]

dp_smc_cost["cost.smc_DP"] = dp_smc_cost["cost.smc_SPAQ"] * ((3.39+0.22)/3.39)

dp_smc_cost["cost.smc.ref_DP"] = dp_smc_cost["cost.smc.ref_SPAQ"] * ((3.39+0.22)/3.39)
dp_smc_cost["cost.smc.low_DP"] = dp_smc_cost["cost.smc.ref_DP"] + (10378.949999999999-6590.175)*0.24

dp_smc_cost["cost_DP"] = dp_smc_cost["cost.RDT_DP"]+dp_smc_cost["cost.ACT_DP"]+dp_smc_cost["cost.facility_DP"]+\
dp_smc_cost["cost.severe_DP"]+dp_smc_cost["cost.smc_DP"]+dp_smc_cost["cost.bednet_DP"]
dp_smc_cost["cost.low_DP"] = dp_smc_cost["cost.RDT_DP"]+dp_smc_cost["cost.ACT_DP"]+dp_smc_cost["cost.facility_DP"]+\
dp_smc_cost["cost.severe_DP"]+dp_smc_cost["cost.smc.low_DP"]+dp_smc_cost["cost.bednet_DP"]

In [71]:
print(dp_smc_cost["cost.RDT_DP"]+dp_smc_cost["cost.ACT_DP"]+dp_smc_cost["cost.facility_DP"]+dp_smc_cost["cost.severe_DP"])

print(dp_smc_cost["cost.RDT_SPAQ"]+dp_smc_cost["cost.ACT_SPAQ"]+dp_smc_cost["cost.facility_SPAQ"]+dp_smc_cost["cost.severe_SPAQ"])

0    25859.400361
1    36696.943379
2    45385.774023
3    56900.383539
dtype: float64
0    28802.914586
1    41259.957188
2    49648.026811
3    60799.274803
dtype: float64


In [72]:
print(dp_smc_cost["cost.smc_DP"])

print(dp_smc_cost["cost.smc_SPAQ"])

print(dp_smc_cost["cost.smc_DP"]/dp_smc_cost["cost.smc_SPAQ"])

print(((3.39+0.22)/3.39))

0    148360.963109
1    154799.619162
2    145396.915050
3    150402.870364
Name: cost.smc_DP, dtype: float64
0    139319.574776
1    145365.847357
2    136536.161224
3    141237.044469
Name: cost.smc_SPAQ, dtype: float64
0    1.064897
1    1.064897
2    1.064897
3    1.064897
dtype: float64
1.064896755162242


In [73]:
print(dp_smc_cost["cost.low_DP"]/5)
print(dp_smc_cost["cost.low_SPAQ"]/5)

0    27294.670647
1    30171.829233
2    30317.668059
3    33048.150932
Name: cost.low_DP, dtype: float64
0    29567.169732
1    32891.402001
2    32990.222074
3    35787.928920
Name: cost.low_SPAQ, dtype: float64


In [57]:
dp_smc_cost["cost_DP"]/5

0    37617.939895
1    41073.179709
2    40930.405015
3    44234.517981
Name: cost_DP, dtype: float64

In [58]:
dp_smc_cost["cost_SPAQ"]/5

0    36398.365073
1    40099.028109
2    40010.704808
3    43181.131055
Name: cost_SPAQ, dtype: float64

In [48]:
ipt_cost["cost"]/5

0    25801.699988
1    28934.790248
2    28693.693555
3    30935.246570
Name: cost, dtype: float64

u10 SMC DP results: (for 25% pfpr2-10 baseline)
- burden averted: = 21%
- cost per 1000 (high) = \$40,930
- cost per 1000 (low) = \$30,320


# u15 SMC

In [75]:
df_u5 = foo[foo["scenario_number"]==30].reset_index(drop=True)
df_u10 = foo[foo["scenario_number"]==35].reset_index(drop=True)
df_u15 = foo[foo["scenario_number"]==36].reset_index(drop=True)
df_u10_dp = foo[foo["scenario_number"]==46].reset_index(drop=True)
df_u15_dp = foo[foo["scenario_number"]==47].reset_index(drop=True)
print(np.mean(df_u5["Received_SMC"]))
print(np.mean(df_u10["Received_SMC"]))
print(np.mean(df_u15["Received_SMC"]))

df_u15_dp["case_ratio_to_old"] = df_u15_dp["clinical_incidence_all"]/df_u15["clinical_incidence_all"]
df_u15_dp["severe_case_ratio_to_old"] = df_u15_dp["severe_incidence_all"]/df_u15["severe_incidence_all"]

6590.175
10378.949999999999
14450.675


In [76]:
(df_u5["clinical_incidence_all"]-df_u15_dp["clinical_incidence_all"])/df_u5["clinical_incidence_all"]

0    0.483158
1    0.445174
2    0.383643
3    0.310172
Name: clinical_incidence_all, dtype: float64

In [80]:
ipt_cost = marita_condensed[marita_condensed["scenario_number"]==13].reset_index(drop=True)
spaq_smc_cost = marita_condensed[marita_condensed["scenario_number"]==36].reset_index(drop=True)
dp_smc_cost = spaq_smc_cost.copy(deep=True)

In [81]:
dp_smc_cost = dp_smc_cost.merge(spaq_smc_cost, on="Scenario", suffixes=("_DP", "_SPAQ"))

In [82]:
dp_smc_cost["scenario_number_DP"] = 47
dp_smc_cost["cost.RDT_DP"] = dp_smc_cost["cost.RDT_SPAQ"]*df_u15_dp["case_ratio_to_old"]
dp_smc_cost["cost.ACT_DP"] = dp_smc_cost["cost.ACT_SPAQ"]*df_u15_dp["case_ratio_to_old"]
dp_smc_cost["cost.facility_DP"] = dp_smc_cost["cost.facility_SPAQ"]*df_u15_dp["case_ratio_to_old"]
dp_smc_cost["cost.severe_DP"] = dp_smc_cost["cost.severe_SPAQ"]*df_u15_dp["severe_case_ratio_to_old"]

dp_smc_cost["cost.smc_DP"] = dp_smc_cost["cost.smc_SPAQ"] * ((3.39+0.22)/3.39)

dp_smc_cost["cost.smc.ref_DP"] = dp_smc_cost["cost.smc.ref_SPAQ"] * ((3.39+0.22)/3.39)
dp_smc_cost["cost.smc.low_DP"] = dp_smc_cost["cost.smc.ref_DP"] + (14450.675-6590.175)*0.24

dp_smc_cost["cost_DP"] = dp_smc_cost["cost.RDT_DP"]+dp_smc_cost["cost.ACT_DP"]+dp_smc_cost["cost.facility_DP"]+\
dp_smc_cost["cost.severe_DP"]+dp_smc_cost["cost.smc_DP"]+dp_smc_cost["cost.bednet_DP"]
dp_smc_cost["cost.low_DP"] = dp_smc_cost["cost.RDT_DP"]+dp_smc_cost["cost.ACT_DP"]+dp_smc_cost["cost.facility_DP"]+\
dp_smc_cost["cost.severe_DP"]+dp_smc_cost["cost.smc.low_DP"]+dp_smc_cost["cost.bednet_DP"]

In [83]:
print(dp_smc_cost["cost.RDT_DP"]+dp_smc_cost["cost.ACT_DP"]+dp_smc_cost["cost.facility_DP"]+dp_smc_cost["cost.severe_DP"])

print(dp_smc_cost["cost.RDT_SPAQ"]+dp_smc_cost["cost.ACT_SPAQ"]+dp_smc_cost["cost.facility_SPAQ"]+dp_smc_cost["cost.severe_SPAQ"])

0    17149.710576
1    26655.286024
2    36014.051831
3    49210.411271
dtype: float64
0    22486.250118
1    34441.791344
2    43240.027708
3    55886.986769
dtype: float64


In [84]:
print(dp_smc_cost["cost.smc_DP"])

print(dp_smc_cost["cost.smc_SPAQ"])

print(dp_smc_cost["cost.smc_DP"]/dp_smc_cost["cost.smc_SPAQ"])

print(((3.39+0.22)/3.39))

0    203920.438018
1    217255.481863
2    207370.912355
3    205088.642868
Name: cost.smc_DP, dtype: float64
0    191493.153706
1    204015.535600
2    194733.349829
3    192590.166017
Name: cost.smc_SPAQ, dtype: float64
0    1.064897
1    1.064897
2    1.064897
3    1.064897
dtype: float64
1.064896755162242


In [85]:
print(dp_smc_cost["cost.low_DP"]/5)

0    25748.175490
1    28358.940562
2    28638.766420
3    31705.599279
Name: cost.low_DP, dtype: float64


In [86]:
print(dp_smc_cost["cost_DP"]/5)

0    46987.896919
1    51556.020778
2    51450.860038
3    53633.678028
Name: cost_DP, dtype: float64


u15 SMC DP results: (for 25% pfpr2-10 baseline)
- burden averted: = 38%
- cost per 1000 (high) = \$51,450
- cost per 1000 (low) = \$28,640